In [36]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
all_data = num_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [37]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words= stopwords_pt)

cv.fit(neg_train["texts"])
vocab_neg = set(cv.vocabulary_.keys())

cv.fit(pos_train["texts"])
vocab_pos = set(cv.vocabulary_.keys())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2630
Dimensão vocabulario positivo: 2272


In [38]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))
print("Dimensão: ", len(lexicon))

Dimensão:  2734


In [39]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = False))
                    ])

print("USING FEATURE FREQUENCY")
evaluate(all_data, features, 10)

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = True))
                    ])
print("USING FEATURE PRESENCE")
evaluate(all_data, features, 10)

USING FEATURE FREQUENCY
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.815955734406
Desvio padrão:  0.0208149938779

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.824527162978
Desvio padrão:  0.0256455898406

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.781750503018
Desvio padrão:  0.0460324225265
USING FEATURE PRESENCE
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.813118712274
Desvio padrão:  0.01860809145

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.827384305835
Desvio padrão:  0.0281973772125

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.790301810865
Desvio padrão:  0.0478120064414


In [43]:
# Avaliando a incidência dos bigramas
cv = CountVectorizer(ngram_range=(1,2), stop_words= stopwords_pt)

neg_counts = cv.fit_transform(neg_train["texts"])
neg_bigrams = cv.vocabulary_

pos_counts = cv.fit_transform(pos_train["texts"])
pos_bigrams = cv.vocabulary_

sp = np.sum(pos_counts.toarray(), axis=0)
sn = np.sum(neg_counts.toarray(), axis=0)

pos_bigrams  = {bigram: sp[index] for bigram, index in pos_bigrams.items()}
neg_bigrams  = {bigram: sn[index] for bigram, index in neg_bigrams.items()}

pos_excl_bigrams = {key:value for key, value in pos_bigrams.items() if key not in neg_bigrams}
neg_excl_bigrams = {key:value for key, value in neg_bigrams.items() if key not in pos_bigrams}

lexicon_bigrams = list(pos_excl_bigrams.keys()) + list(neg_excl_bigrams.keys())
len(lexicon_bigrams)

12791

In [44]:
from pandas import Series
s = Series(neg_excl_bigrams)
s.nlargest(15)

minas energia           15
edison                  12
edison lobão            12
sabia                   12
alves                   11
sabesp                  11
alves pmdb              10
câmara                  10
eduardo alves           10
henrique eduardo        10
morto                   10
presidente república    10
vaccari                 10
calheiros                9
envolvidos esquema       9
dtype: int64

In [46]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

print("USING FEATURE FREQUENCY")
evaluate(all_data, features, 10)

print("\nUSING FEATURE PRESENCE")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams, binary= True))
                    ])
evaluate(all_data, features, 10)

USING FEATURE FREQUENCY
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.811670020121
Desvio padrão:  0.0169219957015

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.848732394366
Desvio padrão:  0.0335021600879

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.756016096579
Desvio padrão:  0.0360915769881

USING FEATURE PRESENCE
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.811670020121
Desvio padrão:  0.0180878485468

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.847303822938
Desvio padrão:  0.0363430023044

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.765975855131
Desvio padrão:  0.0451019803645
